In [1]:
import os
import glob
import nibabel as nib
import numpy as np
import enum
import json

In [2]:
bids_dir = "bids-new"

In [3]:
session_dirs = []
for json_path in sorted(glob.glob(os.path.join(bids_dir, "sub*", "ses*", "anat", "*echo-01*mag*json"))):
    with open(json_path, 'r') as json_file:
        json_data = json.load(json_file)
        if json_data['ProtocolName'] == "t2starME_qsm_tra_Iso1.4mm_INPHASE_bipolar_RUN_THIS_ONE":
            session_dirs.append(os.sep.join(os.path.split(json_path)[0].split(os.sep)[:-1]))
print(f"{len(session_dirs)} sessions found:")
for session_dir in session_dirs: print(session_dir)

15 sessions found:
bids-new/sub-z0034542/ses-20220715
bids-new/sub-z0186251/ses-20221107
bids-new/sub-z0705200/ses-20230104
bids-new/sub-z0755228/ses-20211108
bids-new/sub-z1167038/ses-20220315
bids-new/sub-z1181657/ses-20220315
bids-new/sub-z1262112/ses-20220314
bids-new/sub-z1472355/ses-20221222
bids-new/sub-z1728751/ses-20220328
bids-new/sub-z1778013/ses-20220715
bids-new/sub-z1818796/ses-20230313
bids-new/sub-z2007565/ses-20220715
bids-new/sub-z2904752/ses-20220826
bids-new/sub-z3171177/ses-20230313
bids-new/sub-z3278008/ses-20211109


In [4]:
qsm_files = [glob.glob(os.path.join(session_dir, "extra_data", "*qsm.*")) for session_dir in session_dirs]
seg_files = [glob.glob(os.path.join(session_dir, "extra_data", "*segmentation.*")) for session_dir in session_dirs]

qsm_files = [qsm_file_list[0] for qsm_file_list in qsm_files if len(qsm_file_list)]
seg_files = [seg_file_list[0] for seg_file_list in seg_files if len(seg_file_list)]

print(f"{len(qsm_files)} QSM images found.")
print(f"{len(seg_files)} segmentations found.")

class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

for i in range(len(qsm_files)):
    # load files
    qsm_nii = nib.load(qsm_files[i])
    seg_nii = nib.load(seg_files[i])

    # get image data
    qsm = qsm_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # separate prostate tissue values
    prostate_values = qsm[seg == SegType.PROSTATE.value]

    # identify susceptibility values less than two standard deviations below the mean
    inliers = qsm + 3*np.std(prostate_values) > np.mean(qsm)

    # clean up segmentations such that 'inliers' are excluded and prostate is removed
    seg[np.logical_and(seg == SegType.GOLD_SEED.value, inliers)] = 0
    seg[np.logical_and(seg == SegType.CALCIFICATION.value, inliers)] = 0
    seg[seg == SegType.PROSTATE.value] = 0
    seg[seg == SegType.GOLD_SEED.value] = 1
    seg[seg == SegType.CALCIFICATION.value] = 2

    # save result using original file extension
    extension = ".".join(seg_files[i].split('.')[1:])
    filename=f"{seg_files[i].split('.')[0]}_clean.{extension}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)

    seg_files[i] = filename

15 QSM images found.
15 CT images found.
15 segmentations found.


/home/uqaste15/miniconda3/envs/prostate/lib/python3.8/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
